In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
from fbprophet import Prophet
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
avocado = pd.read_csv("../input/avocado-prices/avocado.csv")
avocado.head()

Данные представляют собой статистику продаж авокадо, опубликованную на сайте Hass Avocado Board в мае 2018. В ней присутствут информация о средней цене авокадо ('AveragePrice'), и количестве проданных за неделю авокадо с PLU4046 ('4046'), c PLU4225 ('4225'), c PLU4770 ('4770') и всех вместе ('Total Volume'). Данные для классов 'conventional' и 'organic' собраны раздельно (за это отвечает колонка 'type'). Сбор данных происходил в 54 регионах США (за регион отвечает колонка 'region') c января 2015 по март 2018 с частотой в одну неделю (за время, к которому относятся данные, отвечает колонка 'Date').

В качестве предобработки данных разобьем conventional и organic по разным таблицам. Также, поскольку мы будем рассматривать статистику по всем регионам в совокупности, то данные нужно будет объединить: для количеств достаточно просуммировать по всем регионам (однако, из-за того, что сумарное количество окажется очень большим, разумно будет сменить базовую единицу измерения с "1 фрукт" на "1000 фруктов"), средняя цена - отношение суммарного количества потраченных на покупку авокадо денег (для каждого отдельного региона - это произведение средней цены на объем продаж) к суммарному объему продаж. Таким образом мы получим две таблицы, в каждой из которых дату измерения можно взять в качестве индекса.

In [ ]:
conventional = avocado[avocado.type=='conventional'].drop(columns=['type', 'year', 'Unnamed: 0'])
conventional.head()

In [ ]:
organic = avocado[avocado.type=='organic'].drop(columns=['type', 'year', 'Unnamed: 0'])
organic.head()

In [ ]:
date = list(avocado.Date.unique())
date.sort()

In [ ]:
total_conventional = pd.DataFrame({'Date': [datetime.date(int(d[0:4]), int(d[5:7]), int(d[8:10])) for d in date],
                                  'AveragePrice': [(conventional[conventional.Date == d].AveragePrice*conventional[conventional.Date == d]['Total Volume']).sum()/conventional[conventional.Date == d]['Total Volume'].sum() for d in date],
                                  'Total Volume': [conventional[conventional.Date == d]['Total Volume'].sum()/1000 for d in date],
                                  '4046': [conventional[conventional.Date == d]['4046'].sum()/1000 for d in date],
                                  '4225': [conventional[conventional.Date == d]['4225'].sum()/1000 for d in date],
                                  '4770': [conventional[conventional.Date == d]['4770'].sum()/1000 for d in date]}).set_index('Date')
total_conventional.head()

In [ ]:
total_organic = pd.DataFrame({'Date': [datetime.date(int(d[0:4]), int(d[5:7]), int(d[8:10])) for d in date],
                                  'AveragePrice': [(organic[organic.Date == d].AveragePrice*organic[organic.Date == d]['Total Volume']).sum()/organic[organic.Date == d]['Total Volume'].sum() for d in date],
                                  'Total Volume': [organic[organic.Date == d]['Total Volume'].sum()/1000 for d in date],
                                  '4046': [organic[organic.Date == d]['4046'].sum()/1000 for d in date],
                                  '4225': [organic[organic.Date == d]['4225'].sum()/1000 for d in date],
                                  '4770': [organic[organic.Date == d]['4770'].sum()/1000 for d in date]}).set_index('Date')
total_organic.head()

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries, window = 48, cutoff = 0.05):

    rolmean = timeseries.rolling(window).mean()
    rolstd = timeseries.rolling(window).std()

    fig = plt.figure(figsize=(12, 4))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    fig = plt.figure(figsize=(12, 4))
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    
    plt.show()
    
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries.values,autolag='AIC' )
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    pvalue = dftest[1]
    if pvalue < cutoff:
        print('p-value = %.4f. The series is likely stationary.' % pvalue)
    else:
        print('p-value = %.4f. The series is likely non-stationary.' % pvalue)
    
    print(dfoutput)

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_conventional.index, total_conventional.AveragePrice, 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_conventional['AveragePrice'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_conventional['AveragePrice'], window=48)

Из анализа сезонности следует, что цена авокадо класса 'conventional' зимой наиболее низкая, весной и летом она растет, достигая пика к середине осени, после чего происходит резкий спад. При этом присутствует внесезонный тренд на подорожание. 

Так как связанный с ценой авокадо временной ряд не стационарен, продифференцируем его:

In [ ]:
d_c_ap = (total_conventional['AveragePrice']-total_conventional['AveragePrice'].shift(1)).dropna()
sm.tsa.seasonal_decompose(d_c_ap, model='additive',period= 48).plot()
plt.show()
test_stationarity(d_c_ap, window=48)

Недельный прирост цены, напротив, является стационарным. Его скользящее среднее почти всегда положительно и колеблется между 0 и 0.01 из чего следует, что цена стабильно растет. 

Сезонная составляющая прироста цены почти всегда незначительна и играет роль только в конце осени, когда у неё имеется отрицательный пик (обеспечивающий сезонное подешевение авокадо в конце осени, но недостаточно сильный, чтобы переломить общий тренд на подорожание). 

Предскажем цену на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_conventional.index, 'y': total_conventional['AveragePrice']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis= 1) ,train_df['y'])
fig = plt.figure(figsize=(12, 4))
forecast = model.predict(test_df)
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_conventional.index, total_conventional['Total Volume'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_conventional['Total Volume'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_conventional['Total Volume'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'conventional' в конце осени наиболее низкое, в зимой начинает быстро расти, достигает пика в начале весны, после чего медленно идёт на спад (при этом наблюдается втрой пик поменьше в районе мая). Временной ряд стационарен, а скользящее среднее колеблется между 85 и 90 миллионами фруктов.

Предскажем количество проданных за неделю авокадо класса 'conventional' на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_conventional.index, 'y': total_conventional['Total Volume']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_conventional.index, total_conventional['4046'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_conventional['4046'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_conventional['4046'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'conventional' с PLU4046 в конце осени / начале зимы наиболее низкое, в зимой начинает быстро расти, достигает пика в начале весны, после чего медленно идёт на спад (при этом наблюдается втрой пик поменьше в районе мая). Временной ряд стационарен, а скользящее среднее колеблется между 30 и 32 миллионами фруктов.

Предскажем количество проданных за неделю авокадо класса 'conventional' с PLU4046 на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_conventional.index, 'y': total_conventional['4046']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_conventional.index, total_conventional['4225'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_conventional['4225'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_conventional['4225'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'conventional' с PLU4225 в конце осени / начале зимы наиболее низкое, в зимой начинает быстро расти, достигает пика в начале весны, после чего медленно идёт на спад (при этом наблюдается втрой пик поменьше в районе мая). 

Временной ряд не стационарен, а потому продифференцируем его:

In [ ]:
d_c_4225 = (total_conventional['4225']-total_conventional['4225'].shift(1)).dropna()
sm.tsa.seasonal_decompose(d_c_4225, model='additive',period= 48).plot()
plt.show()
test_stationarity(d_c_4225, window=48)

Недельный прирост, напротив, является стационарным. Его скользящее среднее почти всегда близко к 0, а потому изменения этого показателя носят в основном сезонный характер.

Сезонная составляющая прироста представлена в основном изменением дисперсии, которая достигает минимума в январе и максимума в марте.

Предскажем количество проданных за неделю авокадо класса 'conventional' с PLU4225 на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_conventional.index, 'y': total_conventional['4225']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_conventional.index, total_conventional['4770'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_conventional['4770'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_conventional['4770'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'conventional' с PLU4770 в конце осени / начале зимы наиболее низкое, в зимой начинает быстро расти, достигает пика в начале весны, после чего медленно идёт на спад.

Временной ряд не стационарен, а потому продифференцируем его:

In [ ]:
d_c_4770 = (total_conventional['4770']-total_conventional['4770'].shift(1)).dropna()
sm.tsa.seasonal_decompose(d_c_4770, model='additive',period= 48).plot()
plt.show()
test_stationarity(d_c_4770, window=48)

Недельный прирост, напротив, является стационарным. Его скользящее среднее почти всегда близко к 0, а потому изменения этого показателя носят в основном сезонный характер.

Сезонная составляющая прироста представлена в основном изменением дисперсии, которая достигает минимума в январе и максимума в марте.

Предскажем количество проданных за неделю авокадо класса 'conventional' с PLU4770 на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_conventional.index, 'y': total_conventional['4770']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_organic.index, total_organic.AveragePrice, 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_organic['AveragePrice'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_organic['AveragePrice'], window=48)

Из анализа сезонности следует, что цена авокадо класса 'organic' в начале весны наиболее низкая, весной и летом она растет, достигая пика к середине осени, после чего идет на спад. Временной ряд стационарен, а скользящее среднее колеблется между 1.5 и 1.7.

Предскажем цену на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_organic.index, 'y': total_organic['AveragePrice']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_organic.index, total_organic['Total Volume'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_organic['Total Volume'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_organic['Total Volume'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'organic' в осенью наиболее низкое, зимой начинает расти, достигает пика в районе мая, после чего идёт на спад.

Временной ряд не стационарен, а потому продифференцируем его:

In [ ]:
d_o_tv = (total_organic['Total Volume']-total_organic['Total Volume'].shift(1)).dropna()
sm.tsa.seasonal_decompose(d_o_tv, model='additive',period= 48).plot()
plt.show()
test_stationarity(d_o_tv, window=48)

Недельный прирост, напротив, является стационарным. Его скользящее среднее почти всегда положительно и колеблется между 0 и 25, из чего следует, что исходный показатель стабильно растет.

Сезонная составляющая прироста представлена в основном изменением дисперсии, которая достигает минимума в январе и максимума в марте.

Предскажем количество проданных за неделю авокадо класса 'organic' на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_organic.index, 'y': total_organic['Total Volume']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_organic.index, total_organic['4046'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_organic['4046'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_organic['4046'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'organic' с PLU4064 в осенью наиболее низкое, зимой начинает расти, достигает пика в районе мая, после чего идёт на спад.

Временной ряд не стационарен, а потому продифференцируем его:

In [ ]:
d_o_4046 = (total_organic['4046']-total_organic['4046'].shift(1)).dropna()
sm.tsa.seasonal_decompose(d_o_4046, model='additive',period= 48).plot()
plt.show()
test_stationarity(d_o_tv, window=48)

Недельный прирост, напротив, является стационарным. Его скользящее среднее почти всегда отрицательно и колеблется между -10 и 0, из чего следует, что исходный показатель стабильно снижается.

Сезонная составляющая прироста представлена в основном изменением дисперсии, которая достигает минимума в январе и максимума в апреле.

Предскажем количество проданных за неделю авокадо класса 'organic' с PLU4064 на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_organic.index, 'y': total_organic['4046']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_organic.index, total_organic['4225'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_organic['4225'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_organic['4225'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'organic' с PLU4225 в осенью наиболее низкое, зимой начинает расти, достигает пика в районе марта, после чего идёт на спад.

Временной ряд не стационарен, а потому продифференцируем его:

In [ ]:
d_o_4225 = (total_organic['4225']-total_organic['4225'].shift(1)).dropna()
sm.tsa.seasonal_decompose(d_o_4225, model='additive',period= 48).plot()
plt.show()
test_stationarity(d_o_tv, window=48)

Недельный прирост, напротив, является стационарным. Его скользящее среднее почти всегда близко к 0, а потому изменения этого показателя носят в основном сезонный характер.

Сезонная составляющая прироста представлена в основном изменением дисперсии, которая достигает минимума в сентябре и максимума в мае.

Предскажем количество проданных за неделю авокадо класса 'conventional' с PLU4225 на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_organic.index, 'y': total_organic['4225']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(total_organic.index, total_organic['4770'], 'k-')
plt.show()

In [ ]:
sm.tsa.seasonal_decompose(total_organic['4770'], model='additive',period= 48).plot()
plt.show()

In [ ]:
test_stationarity(total_organic['4770'], window=48)

Из анализа сезонности следует, что количество проданных за неделю авокадо класса 'organic' с PLU4770 в осенью наиболее низкое, зимой начинает расти, достигает пика в районе мая, после чего идёт на спад.

Временной ряд не стационарен, а потому продифференцируем его:

In [ ]:
d_o_4770 = (total_organic['4770']-total_organic['4770'].shift(1)).dropna()
sm.tsa.seasonal_decompose(d_o_4046, model='additive',period= 48).plot()
plt.show()
test_stationarity(d_o_tv, window=48)

Недельный прирост, напротив, является стационарным. Его скользящее среднее почти всегда отрицательно и колеблется между -10 и 0, из чего следует, что исходный показатель стабильно снижается.

Сезонная составляющая прироста представлена в основном изменением дисперсии, которая достигает минимума в январе и максимума в апреле.

Предскажем количество проданных за неделю авокадо класса 'organic' с PLU4770 на год вперед (до марта 2019 года):

In [ ]:
df_ = pd.DataFrame({'ds': total_organic.index, 'y': total_organic['4770']}).dropna()
prophet_basic = Prophet()
prophet_basic.fit(df_)
future= prophet_basic.make_future_dataframe(periods= 48, freq = 'W', include_history = False)
future['label'] = 'test'
future['y'] = np.nan
df_['label'] = 'train'
df = pd.concat((df_,future), axis = 0)
features = []
for period in range(48,96,4):
    df["lag_period_{}".format(period)] = df.y.shift(period)
    features.append("lag_period_{}".format(period))
df['lagf_mean'] = df[features].mean(axis = 1)
features.extend(['lagf_mean'])
model = XGBRegressor()
train_df = df[df.label == 'train'][features + ['y']].dropna()
test_df = df[df.label == 'test'][features]
model.fit(train_df.drop('y', axis = 1) ,train_df['y'])
forecast = model.predict(test_df)
fig = plt.figure(figsize=(12, 4))
plt.plot(future['ds'], forecast, 'k-')
plt.show()

Из всего этого можно сделать следующие выводы: 

Первый вывод касается сезонного цикла авокадо. По данным видно, что у объема продаж авокадо как правило два пика - в марте и в мае (они, скорее всего, соответствуют времени созревания авокадо на восточном и западном побережье (Флориде и Калифорнии соответственно)). При этом период наибольшего дефицита авокадо приходится на конец осени, начало зимы (когда прошлогодние авокадо уже закончились, а новые ещё не созрели). Цена же, напротив достигает максимума в конце-лета / начале осени (что скорее всего, связано с тем, что во Флориде закончился сезон, а спрос остался прежним), а минимума в конце осени (спрос падает из-за появления на рынке других фруктов, сезон которых приходится на это время).

Второй вывод касается внесезонных трендов. Повсеместно наблюдается уменьшение продаж авокадо наиболее распространенных сортов ('PLU4046', 'PLU4225' и 'PLU4770'). При этом общее количество продаж не уменьшается ('conventional') или даже растет ('organic'), что свидетельствует о росте объема продаж более редких сортов авокадо, статистика по которым здесь, к сожалению, не представлена. Также наблюдается рост цен на авокадо класса 'conventional', связанный либо с ростом спроса при неизменности предложения, либо с заменой более дешевых сортов авокадо более дорогими. При этом цены на авокадо класса 'organic' стабильны, так как вышеперечисленные факторы в этом случае компенсируются ростом предложения.